In [1]:
from ctypes import *
from ctypes.wintypes import *
import numpy as np
import psutil
import sys
from util.player import player

In [2]:
NAME_OF_PROCESS = "fm.exe"
PROCESS_ID = None

PLAYERS_START_ADDR = 0x04f26000
OFFSET = 0x20
START_ADDR = PLAYERS_START_ADDR + OFFSET

PLAYER_ADDR_STRLEN = 4

PROCESS_VM_READ = 0x0010

PLAYER_CLASS_ID = 0x0133f1a80133ec44

CURRENT_DATE_YEAR = None
CURRENT_DATE_DAYS = None

In [3]:
k32 = WinDLL('kernel32')
k32.OpenProcess.argtypes = DWORD,BOOL,DWORD
k32.OpenProcess.restype = HANDLE
k32.ReadProcessMemory.argtypes = HANDLE,LPVOID,LPVOID,c_size_t,POINTER(c_size_t)
k32.ReadProcessMemory.restype = BOOL

In [36]:
# find pid
def set_pid():
    global PROCESS_ID
    for proc in psutil.process_iter():
        if proc.name() == NAME_OF_PROCESS:
            PROCESS_ID = proc.pid
            break
        
def set_current_date():
    
    global CURRENT_DATE_YEAR
    global CURRENT_DATE_DAYS
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(4)
    s = c_size_t()
    
    current_date_addr = 0x0163869c
    
    if k32.ReadProcessMemory(process, current_date_addr, buf, 4, byref(s)):
        reverse_array(buf)
        CURRENT_DATE_YEAR = int(buf.raw.hex()[0:4],16)
        CURRENT_DATE_DAYS = int(buf.raw.hex()[4:8],16)
    else:
        print("set_current_date: Access Denied!")

def reverse_array(array):
    length = len(array)
    left = 0
    right = length-1
    while left < right:
        temp = array[left]
        array[left] = array[right]
        array[right] = temp
        left = left + 1
        right = right - 1

        
def reverse_hex_string(st):
    length = len(st)
    st = list(st)
    left = 0
    right = length
    while left < right:
        temp = st[left:left+2]
        st[left:left+2] = st[right-2:right]
        st[right-2:right] = temp
        left = left + 2
        right = right - 2
    return "".join(st)
        
def decode_player(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(PLAYER_ADDR_STRLEN)
    s = c_size_t()
    
    uid_addr = addr + 232
    
    if k32.ReadProcessMemory(process, uid_addr, buf, 4, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1
    
def decode_player_class_id(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(8)
    s = c_size_t()
    
    if k32.ReadProcessMemory(process, addr, buf, 8, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1   

In [37]:
set_pid()
set_current_date()

In [ ]:
players = []

process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

while True:
    
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        player_addr = int(buf.raw.hex(),16)
        if player_addr == 0:
            break
        class_id = decode_player_class_id(player_addr)
        if class_id == PLAYER_CLASS_ID:
            p = player(PROCESS_ID)
            p.populate_player_from_addr(player_addr)
            players.append(p)
        elif class_id == -1:
            print("Returned -1, breaking")
            break
        addr = addr + PLAYER_ADDR_STRLEN
    else:
        print("Access Denied!")
        break

In [ ]:
players.sort(key=lambda x: x.total, reverse = True)